In [99]:
import numpy as np
import pandas  as pd
import seaborn as sea
import matplotlib.pyplot as plt
import glob
paths = sorted(glob.glob(f'datasets/Cryptocurrency Historical Prices/*.csv'))

In [100]:
dicionario = {'Csv':[],'Instancias':[]}
for i in range(len(paths)):
    dicionario['Csv'].append(paths[i].split('/')[2])
    dicionario['Instancias'].append(pd.read_csv(paths[i]).shape[0])

In [101]:
grupos = pd.DataFrame(dicionario)
grupos

,Csv,Instancias
0,coin_Aave.csv,275
1,coin_BinanceCoin.csv,1442
2,coin_Bitcoin.csv,2991
3,coin_Cardano.csv,1374
4,coin_ChainLink.csv,1385
5,coin_Cosmos.csv,845
6,coin_CryptocomCoin.csv,935
7,coin_Dogecoin.csv,2760
8,coin_EOS.csv,1466
9,coin_Ethereum.csv,2160


In [102]:
g1 = grupos.loc[grupos['Instancias']>2000]
g1

,Csv,Instancias
2,coin_Bitcoin.csv,2991
7,coin_Dogecoin.csv,2760
9,coin_Ethereum.csv,2160
11,coin_Litecoin.csv,2991
12,coin_Monero.csv,2602
13,coin_NEM.csv,2288
16,coin_Stellar.csv,2527
17,coin_Tether.csv,2318
22,coin_XRP.csv,2893


In [103]:
g2 = grupos.loc[(grupos['Instancias']>800) & (grupos['Instancias']<1000)] 
g2

,Csv,Instancias
5,coin_Cosmos.csv,845
6,coin_CryptocomCoin.csv,935
21,coin_WrappedBitcoin.csv,888


In [104]:
g3 = grupos.loc[(grupos['Instancias']<800)] 
g3

,Csv,Instancias
0,coin_Aave.csv,275
14,coin_Polkadot.csv,320
15,coin_Solana.csv,452
20,coin_Uniswap.csv,292


Farei a divisão de 3 grupos 
1. $>$ 2000 instancias
2. $>$ 800 & $<$ 1000 instancias
3. $<$ 800
> Para eviter viés

In [105]:
def Concats( paths , gn ):
    data = pd.DataFrame()
    for path in paths:
        for nomes in gn:
            if nomes in path:
                data =pd.concat([data, pd.read_csv(path)], ignore_index=True)
    return data
        

In [118]:
dataset = pd.read_csv(paths[22])#Concats(paths,g3['Csv'].values)
dataset.head(3)

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,XRP,XRP,2013-08-05 23:59:59,0.005980,0.005613,0.005875,0.005613,0.0,4.387916e+07
1,2,XRP,XRP,2013-08-06 23:59:59,0.005661,0.004629,0.005637,0.004680,0.0,3.659101e+07
2,3,XRP,XRP,2013-08-07 23:59:59,0.004682,0.004333,0.004669,0.004417,0.0,3.453412e+07


&nbsp;

In [119]:
from sklearn.preprocessing   import LabelEncoder,StandardScaler
from sklearn.base            import BaseEstimator,TransformerMixin
from sklearn.pipeline        import Pipeline
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    cross_val_score,
    KFold,
    StratifiedShuffleSplit
    )
from sklearn.metrics import *

In [120]:
from sklearn.linear_model import ElasticNet,Ridge,SGDRegressor

## Pré-processamento

In [121]:
class Padronizacao(BaseEstimator,TransformerMixin):
    def __init__(self,columns:list | str) -> None:
        self.columns = columns 

    def fit(self,X,y=None):
        return self
    
    def transform(self, X:pd.DataFrame):  
        if ( type(self.columns) == list ):
            X[self.columns]   = StandardScaler().fit_transform(X[self.columns])
        else:
            X[[self.columns]] = StandardScaler().fit_transform(X[[self.columns]])
        return X
    
class Dummies(BaseEstimator,TransformerMixin):
    def __init__(self,columns:list | str) -> None:
        self.columns = columns 

    def fit(self,X,y=None):
        return self
    
    def transform(self, X:pd.DataFrame):
        return pd.get_dummies(X,columns=self.columns,dtype=int)

In [122]:
class RemovendoAtributos(BaseEstimator,TransformerMixin):
    def __init__(self,columns:list) -> None:
        self.columns = columns

    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        return X.drop(columns=self.columns,axis=0)

In [111]:
#dataset['Symbol'] = LabelEncoder().fit_transform( dataset['Symbol'].values )

In [131]:
#x = dataset.drop(columns=['Symbol'],axis=0)
#y = dataset['Symbol']
#train ,test = None,None
#for resultf  in StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42).split(x,y):
#    train , test = resultf[0],resultf[1]
#
#train  = dataset.iloc[train,:]
#test   = dataset.iloc[test,:]
xtrain, xtest, ytrain ,ytest = train_test_split(dataset.drop(columns=['Marketcap'],axis=0).iloc[:,1:],dataset['Marketcap'],random_state=42,test_size=0.2)[3]

ValueError: too many values to unpack (expected 4)

In [132]:
pipeline = Pipeline(
    steps=[
        ('Removing',RemovendoAtributos(['SNo','Name','Symbol'])),
        ('Padronizacao',Padronizacao(['High','Low','Open','Close','Volume'])),
        ('Dummies ',Dummies(['Symbol']))
        ]
)
train = pipeline.fit_transform(dataset)


KeyError: "None of [Index(['Symbol'], dtype='object')] are in the [columns]"

In [114]:
xtrain, xtest, ytrain, ytest = (
    train.drop(columns=['Marketcap'],axis=0).iloc[:,1:],
    test.drop(columns=['Marketcap'],axis=0).iloc[:,1:],
    train['Marketcap'],
    test['Marketcap']
)
xtest

,High,Low,Open,Close,Volume,Symbol_0,Symbol_1,Symbol_2,Symbol_3
853,-0.517009,-0.514227,-0.516504,-0.518928,-0.739304,0,0,1,0
720,-0.498560,-0.496712,-0.496422,-0.502397,-0.731788,0,0,1,0
307,-0.495256,-0.491546,-0.493699,-0.495957,-0.189125,0,1,0,0
977,-0.158115,-0.167299,-0.166548,-0.153787,0.099358,0,0,1,0
886,-0.496519,-0.494956,-0.494007,-0.500030,-0.717890,0,0,1,0
...,...,...,...,...,...,...,...,...,...
1277,-0.192977,-0.173420,-0.172547,-0.192555,0.218754,0,0,0,1
512,-0.178766,-0.159847,-0.171478,-0.164814,1.131915,0,1,0,0
1051,-0.492528,-0.492985,-0.492112,-0.498224,-0.070457,0,0,0,1
323,-0.495639,-0.493783,-0.495606,-0.496561,-0.237941,0,1,0,0


In [115]:
Elastic = ElasticNet()
Elastic.fit(xtrain,ytrain)

ElasticNet()

In [116]:
predElastic = Elastic.predict(xtest)

In [117]:
print("Mean Absolute error..:",mean_absolute_error(ytest,predElastic))
print("Mean Squared error...:",np.sqrt(mean_squared_error(ytest,predElastic)))
print("r2 score.............:",r2_score(ytest,predElastic))

Mean Absolute error..: 5124824622.17966
Mean Squared error...: 7030858504.023466
r2 score.............: 0.4669746424143224
